In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

from scipy.stats import trim_mean  
from scipy import stats
from scipy.stats import sem

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
from nltk import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import spacy

In [2]:
from nltk.corpus import brown
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [3]:
fuzz.partial_ratio("this is a test", "this is a test!")

100

# Data Supermercados

In [4]:
df = pd.read_csv('../../../data/datamarket_clean_unique.csv')
df.head()

,Unnamed: 0,url,supermarket,category,name,price,reference_price,reference_unit,product_id,Date,name_lemma,name_lemma_space,name_stemm,name_stemm_space
0,3233,https://www.dia.es/compra-online/productos/beb...,dia-es,bebidas_isotonicas_y_energeticas_isotonicas,POWERADE bebida refrescante aromatizada ice st...,1.39,2.78,l,79d9fe5ead25b3f5a8629a67af0f379e,2021-03-06,poweradirbebidarefrescantearomatizadostormzero...,poweradir bebida refrescante aromatizado storm...,poweradbebrefrescaromatizstormzerbotell,powerad beb refresc aromatiz storm zer botell
1,4593,https://www.carrefour.es/supermercado/pomelo-c...,carrefour-es,el_mercado_frutas_naranjas_y_otros_citricos,Pomelo Círculo de Calidad 1 kg aprox,1.75,1.75,kg,6d8ad09c426320e638209da067f2e5f2,2021-03-06,pomelocírculocalidadaprox,pomelo círculo calidad aprox,pomelcirculcalidaprox,pomel circul calid aprox
2,4680,https://tienda.mercadona.es/product/9522/mini-...,mercadona-es,cereales_y_galletas_cereales,Mini cereales rellenos de chocolate y avellana...,1.30,7.43,kg,527837fde906593fa80458fd7a9b546e,2021-03-06,minicerealrellenochocolateavellanahacendado,mini cereal relleno chocolate avellana hacendado,minicerealrellenchocolatavellanhacend,mini cereal rellen chocolat avellan hacend
3,5854,https://www.carrefour.es/supermercado/baguette...,carrefour-es,el_mercado_panaderia_bolleria_y_pasteleria_pan...,Baguette precocida Carrefour pack de 2 unidade...,0.75,2.50,kg,a06071d024822fcf8e9d12ff897d7b8b,2021-03-06,baguettirprecocidocarrefourpackunidad,baguettir precocido carrefour pack unidad,baguettprecoccarrefourpackunidad,baguett precoc carrefour pack unidad
4,5856,https://www.carrefour.es/supermercado/pan-de-m...,carrefour-es,el_mercado_panaderia_bolleria_y_pasteleria_pan...,Pan de molde sin corteza Natural 100% Bimbo si...,2.30,5.11,kg,7e731ecb83c085e8583683cc228d1423,2021-03-06,moldecortezanaturalbimbolactós,molde corteza natural bimbo lactós,moldcorteznaturalbimblactos,mold cortez natural bimb lactos


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19386 entries, 0 to 19385
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        19386 non-null  int64  
 1   url               19386 non-null  object 
 2   supermarket       19386 non-null  object 
 3   category          19386 non-null  object 
 4   name              19386 non-null  object 
 5   price             19386 non-null  float64
 6   reference_price   19386 non-null  float64
 7   reference_unit    19383 non-null  object 
 8   product_id        19386 non-null  object 
 9   Date              19386 non-null  object 
 10  name_lemma        19384 non-null  object 
 11  name_lemma_space  19384 non-null  object 
 12  name_stemm        19385 non-null  object 
 13  name_stemm_space  19385 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 2.1+ MB


In [6]:
df_carrefour = df[df['supermarket']=='carrefour-es']
df_mercadona = df[df['supermarket']=='mercadona-es']
df_dia = df[df['supermarket']=='dia-es']

# Receta de cocina Test

### Sin Tokenización ni Lemma ni Stemm

In [7]:
# Lentejas estofadas

lentejas_estofadas = '''350 g de lentejas castellanas secas
50 g de pimiento rojo en trozos 3-4 cm
50 g de pimiento verde en trozos 3-4 cm
50 g de zanahoria en trozos 3-4 cm
50 g de cebolleta en trozos 3-4 cm
o bien 50 g de cebolla en trozos 3-4 cm
25 g de aceite de oliva
1 cucharadita de sal
1 cucharadita de pimentón
1 pastilla de caldo de pollo
50 g de chorizo en rodajas
50 g de jamón curado en dados
900 g de agua'''

In [8]:
separador = '\n'

In [9]:
lentejas_estofadas_lst = lentejas_estofadas.split(separador)
lentejas_estofadas_lst

['350 g de lentejas castellanas secas',
 '50 g de pimiento rojo en trozos 3-4 cm',
 '50 g de pimiento verde en trozos 3-4 cm',
 '50 g de zanahoria en trozos 3-4 cm',
 '50 g de cebolleta en trozos 3-4 cm',
 'o bien 50 g de cebolla en trozos 3-4 cm',
 '25 g de aceite de oliva',
 '1 cucharadita de sal',
 '1 cucharadita de pimentón',
 '1 pastilla de caldo de pollo',
 '50 g de chorizo en rodajas',
 '50 g de jamón curado en dados',
 '900 g de agua']

### Con Tokenización, Lemma y Stemm

In [10]:
spanishstemmer=SnowballStemmer('spanish')

In [11]:
nlp = spacy.load('es_core_news_sm')
def normalize_lemma(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    cadena = " ".join(lexical_tokens)
    doc2 = nlp(cadena)
    lemmas = [tok.lemma_.lower() for tok in doc2 if tok.pos_ != 'PRON']
    lemmas_cadena = "".join(lemmas)
    return lemmas_cadena
ingr_lemma = normalize_lemma('350 g de lentejas castellanas secas')
ingr_lemma

'lentejacastellanoseco'

In [12]:
recipe_lemma = []
for i in lentejas_estofadas_lst:
    recipe_lemma.append(normalize_lemma(i))
recipe_lemma

['lentejacastellanoseco',
 'pimientorojotrozo',
 'pimientoverdetrozo',
 'zanahoriartrozo',
 'cebolletartrozo',
 'cebollartrozo',
 'aceiteolivo',
 'cucharaditar',
 'cucharaditarpimentón',
 'pastillacaldopollo',
 'chorizorodaja',
 'jamóncuradodado',
 'agua']

In [13]:
def normalize_lemma_space(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    cadena = " ".join(lexical_tokens)
    doc2 = nlp(cadena)
    lemmas = [tok.lemma_.lower() for tok in doc2 if tok.pos_ != 'PRON']
    lemmas_cadena = " ".join(lemmas)
    return lemmas_cadena

In [14]:
recipe_lemma_space = []
for i in lentejas_estofadas_lst:
    recipe_lemma_space.append(normalize_lemma_space(i))
recipe_lemma_space

['lenteja castellano seco',
 'pimiento rojo trozo',
 'pimiento verde trozo',
 'zanahoriar trozo',
 'cebolletar trozo',
 'cebollar trozo',
 'aceite olivo',
 'cucharaditar',
 'cucharaditar pimentón',
 'pastilla caldo pollo',
 'chorizo rodaja',
 'jamón curado dado',
 'agua']

In [15]:
def normalize_stemm(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    stems = [spanishstemmer.stem(token) for token in lexical_tokens]
    stems_cadena = "".join(stems)
    return stems_cadena
ingr_stemm = normalize_stemm('350 g de lentejas castellanas secas')
ingr_stemm

'lentejcastellansec'

In [16]:
recipe_stemm = []
for i in lentejas_estofadas_lst:
    recipe_stemm.append(normalize_stemm(i))
recipe_stemm

['lentejcastellansec',
 'pimientrojtroz',
 'pimientverdtroz',
 'zanahoritroz',
 'cebollettroz',
 'cebolltroz',
 'aceitoliv',
 'cucharadit',
 'cucharaditpimenton',
 'pastillcaldpoll',
 'chorizrodaj',
 'jamoncurdad',
 'agu']

In [17]:
def normalize_stemm_space(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    stems = [spanishstemmer.stem(token) for token in lexical_tokens]
    stems_cadena = " ".join(stems)
    return stems_cadena

In [18]:
recipe_stemm_space = []
for i in lentejas_estofadas_lst:
    recipe_stemm_space.append(normalize_stemm_space(i))
recipe_stemm_space

['lentej castellan sec',
 'pimient roj troz',
 'pimient verd troz',
 'zanahori troz',
 'cebollet troz',
 'ceboll troz',
 'aceit oliv',
 'cucharadit',
 'cucharadit pimenton',
 'pastill cald poll',
 'choriz rodaj',
 'jamon cur dad',
 'agu']

# Creación de tablas de productos intermedias

### Dia

In [19]:
df_dia.reset_index(drop=True, inplace=True)

In [20]:
df_dia.loc[(df_dia['Date']=='2022-10-16') & 
                 (df_dia['name'].str.lower().str.contains('agua', na=False)),
                 ['name','category', 'product_id', 'price', 'reference_price', 'reference_unit','Date', 'url']]

,name,category,product_id,price,reference_price,reference_unit,Date,url
2423,AQUABONA agua mineral natural botella 1.5 lt,bebidas_aguas,5a0a1bb342bf791eb3296626d0dc88ec,0.59,0.39,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3497,FONT VELLA agua mineral natural botella 75 cl,bebidas_aguas,95eda25354f7042d89e9a5a62b7d0997,0.85,1.13,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3498,PERRIER agua mineral con gas botella 50 cl,bebidas_aguas,e3f82915d0e511d191369e9421816e54,0.99,1.98,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3499,AQUAREL agua mineral natural botella tapón spo...,bebidas_aguas,f55d7e4d9d72a784c5000a5ac289f07e,0.75,1.00,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3500,DIA agua mineral natural botella 1.5 lt,bebidas_aguas,c2790645ef08d7603f9acb25ecc96f70,0.21,0.14,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3501,FONT VELLA agua con gas botella 50 cl,bebidas_aguas,786237374abdc5ac351843ec3f27efca,0.89,1.78,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3503,SAN BENEDETTO agua mineral con frutos rojos ze...,bebidas_aguas,bf17830fc8a382f4acfb6e11e4af2c2d,0.89,2.23,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3524,FONT VELLA agua mineral natural botella 33 cl,bebidas_aguas,d9283465c949711c49f8ef616e1c55bf,0.69,2.09,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3525,DIA agua mineral con gas botella 50 cl,bebidas_aguas,7f7091e11c8f3aefac750483054f2b03,0.21,0.42,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...
3526,URSU9 agua mineral natural alcalina ph9 botell...,bebidas_aguas,ada53c07a9e76b480b39d8a5825a1543,0.69,0.46,l,2022-10-16,https://www.dia.es/compra-online/bebidas/aguas...


In [21]:
df_dia.iloc[3147]['url']

'https://www.dia.es/compra-online/frescos/charcuteria-y-quesos/curados/p/274059'

In [22]:
a_dia = {'b26d51cb5be3aff71a71f87c8908b846' : 'Lentejas', 
         'dada7a323f98cbb8abd3b7a65f11da13' : 'pimiento verde', 
         '3b118605318ccdf76e4db430ef53f94c' : 'pimiento rojo',
         '6d854922c4f8a5a5740991f3e7108da0' : 'zanahoria',
         '06690951566cb9affcb7b886db944225': 'cebolleta dulce', 
         '9dff07cb2f5875147d574f3cd2eb52d7' : 'cebolla',
         'ff4bec68a3103927d5568c80482a6c54' : 'aceite de oliva virgen extra',
         '9e6edc9fb8588e953a8177db9c5fa2c5' : 'aceite de oliva',
         'e453450e28331ea2f9d7b32f423f0ae0': 'sal fina',
         '213ebf1ba8e25da18f50e6770a6fb2bb': 'pimentón',
         '068082a32f9ba83d41c4db0df5087004': 'pastilla de caldo de pollo',
         '616df75e8d66ded901d003e282c3e062' : 'caldo de pollo',
         '26b1a3be138796cd829876975f1d1b01' : 'chorizo',
         '640a8ef50612a09e27d47a75f8ac10f4' : 'jamón curado en dados',
         'c2790645ef08d7603f9acb25ecc96f70' : 'agua'}

### Carrefour

In [23]:
df_carrefour.reset_index(drop=True, inplace=True)

In [24]:
df_carrefour.loc[(df_carrefour['Date']=='2022-10-16') & 
                 (df_carrefour['name'].str.lower().str.contains('pimi', na=False)),
                 ['name','category', 'product_id', 'price', 'reference_price', 'reference_unit','Date', 'url']]

,name,category,product_id,price,reference_price,reference_unit,Date,url
6740,Pimienta negra molida Carrefour 45 g.,la_despensa_alimentacion_especias_y_sazonadores,127a3fc17b8bac1dfdb47ed6438d9e30,0.62,13.78,kg,2022-10-16,https://www.carrefour.es/supermercado/pimienta...
6841,"Relleno para fajitas de pimientos, pollo y ceb...",la_despensa_conservas_sopas_y_precocinados_pla...,0995776c87859933bb0f8b3da70b26b6,1.55,5.17,kg,2022-10-16,https://www.carrefour.es/supermercado/relleno-...
7334,Pimiento del piquillo entero Carrefour 225 g.,la_despensa_conservas_sopas_y_precocinados_con...,004d6b13bdbfa5ba4945c9d645de3af9,0.94,4.18,kg,2022-10-16,https://www.carrefour.es/supermercado/pimiento...
7451,Pimiento del piquillo relleno de bacalao conge...,productos_frescos_pescaderia_preparados_y_elab...,9481892e04c1d46395598662cfa99fc0,4.20,10.50,kg,2022-10-16,https://www.carrefour.es/supermercado/pimiento...
7528,Paté a la pimienta Casa Tarradellas 125 g.,productos_frescos_charcuteria_foie_pates_y_sob...,e0364995a83690a57e419de7379b6216,1.09,8.72,kg,2022-10-16,https://www.carrefour.es/supermercado/pate-a-l...


In [25]:
df_carrefour.iloc[7334]['url']

'https://www.carrefour.es/supermercado/pimiento-del-piquillo-entero-carrefour-225-g/R-526539373/p'

In [26]:
a_carrefour = {'7d751158fe2567a81e991a0a572ce83f' : 'Lentejas', 
               '' : 'pimiento verde', 
               '' : 'pimiento rojo',
               '53e9f777e840fd172ee8bae9747d61f6' : 'zanahoria',
               '452cf4289710f6cb5edb3607fc646185': 'cebolleta dulce', 
               '9d368b1cf54fbb93051b576c09753663' : 'cebolla',
               'ec7cf862264cec9b1bc8344574840325' : 'aceite de oliva virgen extra',
               '276a2380dabc5045fcd8a869f60b88fc' : 'aceite de oliva',
               '': 'sal fina',
               '': 'pimentón',
               '': 'pastilla de caldo de pollo',
               'c33ea2b6f4582b4e4c0579520bb41bcc' : 'caldo de pollo',
               '411367289ea7cb6032db7d65f7286bf4' : 'chorizo',
               '' : 'jamón curado en dados',
               '97dc915b2efecc4be9db3d93635c4c19' : 'agua'}

### Mercadona

In [27]:
df_mercadona.reset_index(drop=True, inplace=True)

In [28]:
df_mercadona.loc[(df_mercadona['Date']=='2022-10-16') & (df_mercadona['name'].str.lower().str.contains('agua mineral grande', na=False)),
                 ['name','category', 'product_id', 'price', 'reference_price', 'reference_unit','Date', 'url']]

,name,category,product_id,price,reference_price,reference_unit,Date,url
1860,Agua mineral grande Bezoya,agua_y_refrescos_agua,7b5c061bb6ad5e8331e04cb847c9cb94,3.96,0.440,l,2022-10-16,https://tienda.mercadona.es/product/27314/agua...
1861,Agua mineral grande Nestlé Aquarel,agua_y_refrescos_agua,e8b2239a25e2b131601db3746e575fd0,1.45,0.290,l,2022-10-16,https://tienda.mercadona.es/product/27052/agua...
1862,Agua mineral grande Nestlé Aquarel,agua_y_refrescos_agua,b66786dccd32c5ce538c2378c1ceeac9,0.54,0.360,l,2022-10-16,https://tienda.mercadona.es/product/27232/agua...
1864,Agua mineral grande Solán de Cabras,agua_y_refrescos_agua,239e582e89b1fe70e40b2875f47999c4,0.79,0.527,l,2022-10-16,https://tienda.mercadona.es/product/27356/agua...
1865,Agua mineral grande Lanjarón,agua_y_refrescos_agua,cca0beef13b0412b6bb6bbf845986163,0.69,0.460,l,2022-10-16,https://tienda.mercadona.es/product/27216/agua...
1866,Agua mineral grande Aguadoy,agua_y_refrescos_agua,19362795c2f1f7a4d87236b5d88a82ab,0.63,0.105,l,2022-10-16,https://tienda.mercadona.es/product/27398/agua...
1867,Agua mineral grande Aguadoy,agua_y_refrescos_agua,c1e1a4c9d0af6984f9c1da705b025be1,0.19,0.127,l,2022-10-16,https://tienda.mercadona.es/product/28472/agua...
1871,Agua mineral grande Fondetal,agua_y_refrescos_agua,72b70a2c8a9f58910067b70e38778184,0.32,0.214,l,2022-10-16,https://tienda.mercadona.es/product/23634/agua...
1874,Agua mineral grande Liviana,agua_y_refrescos_agua,749a61b49de83b28d5a93e87d2ff0976,0.66,0.330,l,2022-10-16,https://tienda.mercadona.es/product/27208/agua...
1889,Agua mineral grande Bezoya,agua_y_refrescos_agua,6bf5ce45fbcc18b2d213edd196b26884,0.66,0.440,l,2022-10-16,https://tienda.mercadona.es/product/27230/agua...


In [29]:
df_mercadona.iloc[1862]['url']

'https://tienda.mercadona.es/product/27232/agua-mineral-grande-nestle-aquarel-mineralizacion-muy-debil-botella'

In [30]:
a_mercadona = {'ecb81249f1dea9150ee04ed7a178029e' : 'Lentejas', 
               'c71150390be4bd6853c9449a8cff62eb' : 'pimiento verde', 
               '43c81c10c1d9eb52b2410de54f274c9d' : 'pimiento rojo',
               'c0ac3da899e51308db2492c91aebbcb2' : 'zanahoria',
               'b2b8c39d71440a4cba62b86f31ce15e7': 'cebolleta dulce', 
               '2f4a5c7049e822ce171768cde7bd2e76' : 'cebolla',
               '488e146ef0d8864d12b7ea721d8de22a' : 'aceite de oliva virgen extra',
               'e4b0fed91e0f81060e98ed076ebe39e4' : 'aceite de oliva',
               '14efd535d750388baa8122354a3bf2ff': 'sal fina',
               '370579b0cb90786d050efc5026103a3f': 'pimentón',
               '74270a2e828834e1dc65088ac8947be5': 'pastilla de caldo de pollo',
               '9161c686d13e241ab7af3f7ef1c191b0' : 'caldo de pollo',
               '4cb684b2f6a44a7d56f7d49b0a265f17' : 'chorizo',
               'f06a69f6ac8f49bfb052af3e0f614998' : 'jamón curado en dados',
               'c1e1a4c9d0af6984f9c1da705b025be1' : 'agua'}

# Probando Localizar un ingrediente

### Método exacto

In [31]:
def get_key(val):
    # list out keys and values separately
    key_list = list(a_mercadona.keys())
    val_list = list(a_mercadona.values())
        
    ratios = process.extractOne(val , a_mercadona)
    if int(ratios[1]) > 80: # can play with this number 
        position = val_list.index(ratios[0])
        return [key_list[position], ratios[0], ratios[1]]
 
    return ['No product_ID', 'No product Match', 'No product Match']

In [32]:
get_key('50 g de pimiento verde en trozos 3-4 cm')

['c71150390be4bd6853c9449a8cff62eb', 'pimiento verde', 90]

In [57]:
def get_recipe_id(recipe_list):
    product_id = []
    match = []
    probabilities = []
    price = []
    exact_price = []
    name = []
    
    for i in recipe_list:
        product_id.append(get_key(i)[0])
        match.append(get_key(i)[1])
        probabilities.append(get_key(i)[2])
        price.append(df_mercadona[df_mercadona.product_id==get_key(i)[0]].price.squeeze())
        name.append(df_mercadona[df_mercadona.product_id==get_key(i)[0]].name.iloc[0])
        
    # buscamos el precio exacto en base al gramaje    
        gr = word_tokenize(i, language='spanish')
        if gr[1] == 'g':
            x = (int(gr[0])/1000) * df_mercadona[df_mercadona.product_id==get_key(i)[0]].reference_price.squeeze()
        elif gr[1] == 'kg':
            x = int(gr[0]) * df_mercadona[df_mercadona.product_id==get_key(i)[0]].reference_price.squeeze()
        elif gr[1] == 'l':
            x = int(gr[0]) * df_mercadona[df_mercadona.product_id==get_key(i)[0]].reference_price.squeeze()
        elif gr[1] == 'cucharadita':
            x = (int(gr[0])*0.005) * df_mercadona[df_mercadona.product_id==get_key(i)[0]].reference_price.squeeze()
        elif gr[1] == 'cucharada':
            x = (int(gr[0])*0.075) * df_mercadona[df_mercadona.product_id==get_key(i)[0]].reference_price.squeeze()
        elif gr[1] == 'pellizco':
            x = (int(gr[0])*0.001) * df_mercadona[df_mercadona.product_id==get_key(i)[0]].reference_price.squeeze()
        else:
            x = 0
        exact_price.append(round(x, 3))
        
    # creamos el dataframe con todos los ingredientes de la receta    
    df = pd.DataFrame({
        'originals': recipe_list,
        'potential matches': match,
        'product_id': product_id,
        'probability of match': probabilities,
        'original_name_exact': name,
        'price_exact': price,
        'price_exact_gr': exact_price})
    return df

In [58]:
px_receta = get_recipe_id(lentejas_estofadas_lst)
px_receta

,originals,potential matches,product_id,probability of match,original_name_exact,price_exact,price_exact_gr
0,350 g de lentejas castellanas secas,Lentejas,ecb81249f1dea9150ee04ed7a178029e,90,Lenteja Hacendado,1.60,0.560
1,50 g de pimiento rojo en trozos 3-4 cm,pimiento rojo,43c81c10c1d9eb52b2410de54f274c9d,90,Pimiento rojo,0.81,0.134
2,50 g de pimiento verde en trozos 3-4 cm,pimiento verde,c71150390be4bd6853c9449a8cff62eb,90,Pimiento verde,0.62,0.114
3,50 g de zanahoria en trozos 3-4 cm,zanahoria,c0ac3da899e51308db2492c91aebbcb2,90,Zanahorias,0.69,0.069
4,50 g de cebolleta en trozos 3-4 cm,cebolleta dulce,b2b8c39d71440a4cba62b86f31ce15e7,86,Cebollas dulces,1.99,0.100
5,o bien 50 g de cebolla en trozos 3-4 cm,cebolla,2f4a5c7049e822ce171768cde7bd2e76,90,Cebollas,1.35,0.000
6,25 g de aceite de oliva,aceite de oliva,e4b0fed91e0f81060e98ed076ebe39e4,90,Aceite de oliva virgen Hacendado,4.20,0.105
7,1 cucharadita de sal,sal fina,14efd535d750388baa8122354a3bf2ff,86,Sal fina Hacendado,0.25,0.001
8,1 cucharadita de pimentón,pimentón,370579b0cb90786d050efc5026103a3f,90,Pimentón dulce Hacendado,0.80,0.071
9,1 pastilla de caldo de pollo,pastilla de caldo de pollo,74270a2e828834e1dc65088ac8947be5,96,Caldo de pollo Hacendado en pastillas,1.00,0.000


In [59]:
print('El precio de esta receta es de:', round(px_receta['price_exact_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra de esta receta es:', round(px_receta['price_exact'].sum(), 2), '€')

El precio de esta receta es de: 2.19 €
El precio de la cesta de la compra de esta receta es: 17.71 €


### Con Lemma

In [36]:
recipe_lemma

['lentejacastellanoseco',
 'pimientorojotrozo',
 'pimientoverdetrozo',
 'zanahoriartrozo',
 'cebolletartrozo',
 'cebollartrozo',
 'aceiteolivo',
 'cucharaditar',
 'cucharaditarpimentón',
 'pastillacaldopollo',
 'chorizorodaja',
 'jamóncuradodado',
 'agua']

In [63]:
def match_lists(list1, list2, recipe):
    matches = []
    probabilities = []
    name = []
    price = []
    exact_price = []

    for i in list1:
        ratios = process.extractOne(i, list2)
        ratios_recipe = process.extractOne(i, recipe)

        if int(ratios[1]) > 50: # can play with this number
            matches.append(ratios[0])
            probabilities.append(ratios[1])
        else:
            matches.append('no match')
            probabilities.append('probability was too low')
            
        name.append(df_mercadona[df_mercadona.name_lemma==ratios[0]].name.iloc[0])
        price.append(df_mercadona[df_mercadona.name_lemma==ratios[0]].price.iloc[0].squeeze())
        
    # buscamos el precio exacto en base al gramaje    
        gr = word_tokenize(ratios_recipe[0], language='spanish')
        if gr[1] == 'g':
            x = (int(gr[0])/1000) * df_mercadona[df_mercadona.name_lemma==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'kg':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_lemma==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'l':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_lemma==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharadita':
            x = (int(gr[0])*0.005) * df_mercadona[df_mercadona.name_lemma==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharada':
            x = (int(gr[0])*0.075) * df_mercadona[df_mercadona.name_lemma==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'pellizco':
            x = (int(gr[0])*0.001) * df_mercadona[df_mercadona.name_lemma==ratios[0]].reference_price.iloc[0].squeeze()
        else:
            x = 0
        exact_price.append(round(x, 3))

    df = pd.DataFrame({
        'originals': lentejas_estofadas_lst,
        'lemma_recipe': list1,
        'potential matches': matches,
        'probability of match': probabilities,
        'original_name_lemma': name,
        'price_lemma': price,
        'price_lemma_gr': exact_price
    }

    )

    return df

In [64]:
df_lemma = match_lists(recipe_lemma, df_mercadona['name_lemma'].tolist(), lentejas_estofadas_lst)
df_lemma

,originals,lemma_recipe,potential matches,probability of match,original_name_lemma,price_lemma,price_lemma_gr
0,350 g de lentejas castellanas secas,lentejacastellanoseco,semilla,64,"Pan con semillas 4,5%",1.50,1.500
1,50 g de pimiento rojo en trozos 3-4 cm,pimientorojotrozo,pimientorojo,83,Pimiento rojo,0.81,0.134
2,50 g de pimiento verde en trozos 3-4 cm,pimientoverdetrozo,pimientoverde,84,Pimiento verde,0.62,0.114
3,50 g de zanahoria en trozos 3-4 cm,zanahoriartrozo,zanahoria,90,Zanahorias,0.85,0.042
4,50 g de cebolleta en trozos 3-4 cm,cebolletartrozo,cebolla,77,Cebollas,1.35,0.068
5,o bien 50 g de cebolla en trozos 3-4 cm,cebollartrozo,cebolla,90,Cebollas,1.35,0.000
6,25 g de aceite de oliva,aceiteolivo,aceiteolivovirgenextraeminente,90,Aceite de oliva virgen extra Eminente,13.00,0.065
7,1 cucharadita de sal,cucharaditar,yuca,68,Yuca,1.91,0.014
8,1 cucharadita de pimentón,cucharaditarpimentón,mento,72,Menta,1.25,0.031
9,1 pastilla de caldo de pollo,pastillacaldopollo,pollo,90,Medio pollo,3.15,0.000


In [62]:
print('El precio de esta receta es de:', round(df_lemma['price_lemma_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra de esta receta es:', round(df_lemma['price_lemma'].sum(), 2), '€')

El precio de esta receta es de: 3.32 €
El precio de la cesta de la compra de esta receta es: 30.57 €


In [40]:
recipe_lemma_space

['lenteja castellano seco',
 'pimiento rojo trozo',
 'pimiento verde trozo',
 'zanahoriar trozo',
 'cebolletar trozo',
 'cebollar trozo',
 'aceite olivo',
 'cucharaditar',
 'cucharaditar pimentón',
 'pastilla caldo pollo',
 'chorizo rodaja',
 'jamón curado dado',
 'agua']

In [65]:
def match_lists_space(list1, list2, recipe):
    matches = []
    probabilities = []
    name = []
    price = []
    exact_price = []

    for i in list1:
        ratios = process.extractOne(i, list2)
        ratios_recipe = process.extractOne(i, recipe)

        if int(ratios[1]) > 50: # can play with this number
            matches.append(ratios[0])
            probabilities.append(ratios[1])
        else:
            matches.append('no match')
            probabilities.append('probability was too low')
            
        name.append(df_mercadona[df_mercadona.name_lemma_space==ratios[0]].name.iloc[0])
        price.append(df_mercadona[df_mercadona.name_lemma_space==ratios[0]].price.iloc[0].squeeze())
        
    # buscamos el precio exacto en base al gramaje    
        gr = word_tokenize(ratios_recipe[0], language='spanish')
        if gr[1] == 'g':
            x = (int(gr[0])/1000) * df_mercadona[df_mercadona.name_lemma_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'kg':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_lemma_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'l':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_lemma_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharadita':
            x = (int(gr[0])*0.005) * df_mercadona[df_mercadona.name_lemma_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharada':
            x = (int(gr[0])*0.075) * df_mercadona[df_mercadona.name_lemma_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'pellizco':
            x = (int(gr[0])*0.001) * df_mercadona[df_mercadona.name_lemma_space==ratios[0]].reference_price.iloc[0].squeeze()
        else:
            x = 0
        exact_price.append(round(x, 3))

    df = pd.DataFrame({
        'originals': lentejas_estofadas_lst,
        'lemma_recipe': list1,
        'potential matches': matches,
        'probability of match': probabilities,
        'original_name_lemma_space': name,
        'price_lemma_space': price,
        'price_lemma_space_gr': exact_price
    }

    )

    return df

In [66]:
df_lemma_space = match_lists_space(recipe_lemma_space, df_mercadona['name_lemma_space'].tolist(), lentejas_estofadas_lst)
df_lemma_space

,originals,lemma_recipe,potential matches,probability of match,original_name_lemma_space,price_lemma_space,price_lemma_space_gr
0,350 g de lentejas castellanas secas,lenteja castellano seco,anís seco regio,86,Anís seco Regio,7.45,2.608
1,50 g de pimiento rojo en trozos 3-4 cm,pimiento rojo trozo,pimiento rojo,95,Pimiento rojo,0.81,0.134
2,50 g de pimiento verde en trozos 3-4 cm,pimiento verde trozo,pimiento verde,95,Pimiento verde,0.62,0.114
3,50 g de zanahoria en trozos 3-4 cm,zanahoriar trozo,zanahoria,90,Zanahorias,0.85,0.042
4,50 g de cebolleta en trozos 3-4 cm,cebolletar trozo,pollo entero filet trozo,86,Pollo entero filetes y trozos,5.48,0.182
5,o bien 50 g de cebolla en trozos 3-4 cm,cebollar trozo,cebolla,90,Cebollas,1.35,0.000
6,25 g de aceite de oliva,aceite olivo,aceite olivo virgen extra eminente,90,Aceite de oliva virgen extra Eminente,13.00,0.065
7,1 cucharadita de sal,cucharaditar,yuca,68,Yuca,1.91,0.014
8,1 cucharadita de pimentón,cucharaditar pimentón,queso madurado cabro pimentón vera villa noble,86,Queso madurado de cabra con pimentón de la Ver...,5.33,0.065
9,1 pastilla de caldo de pollo,pastilla caldo pollo,caldo pollo pastilla,95,Caldo de pollo en pastillas,0.82,0.000


In [67]:
print('El precio de esta receta es de:', round(df_lemma_space['price_lemma_space_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra de esta receta es:', round(df_lemma_space['price_lemma_space'].sum(), 2), '€')

El precio de esta receta es de: 12.51 €
El precio de la cesta de la compra de esta receta es: 42.84 €


### Con Stemm

In [45]:
recipe_stemm

['lentejcastellansec',
 'pimientrojtroz',
 'pimientverdtroz',
 'zanahoritroz',
 'cebollettroz',
 'cebolltroz',
 'aceitoliv',
 'cucharadit',
 'cucharaditpimenton',
 'pastillcaldpoll',
 'chorizrodaj',
 'jamoncurdad',
 'agu']

In [68]:
def match_lists_stemm(list1, list2, recipe):
    matches = []
    probabilities = []
    name = []
    price = []
    exact_price = []

    for i in list1:
        ratios = process.extractOne(i, list2)
        ratios_recipe = process.extractOne(i, recipe)

        if int(ratios[1]) > 50: # can play with this number
            matches.append(ratios[0])
            probabilities.append(ratios[1])
        else:
            matches.append('no match')
            probabilities.append('probability was too low')
            
        name.append(df_mercadona[df_mercadona.name_stemm==ratios[0]].name.iloc[0])
        price.append(df_mercadona[df_mercadona.name_stemm==ratios[0]].price.iloc[0].squeeze())
        
    # buscamos el precio exacto en base al gramaje    
        gr = word_tokenize(ratios_recipe[0], language='spanish')
        if gr[1] == 'g':
            x = (int(gr[0])/1000) * df_mercadona[df_mercadona.name_stemm==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'kg':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_stemm==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'l':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_stemm==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharadita':
            x = (int(gr[0])*0.005) * df_mercadona[df_mercadona.name_stemm==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharada':
            x = (int(gr[0])*0.075) * df_mercadona[df_mercadona.name_stemm==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'pellizco':
            x = (int(gr[0])*0.001) * df_mercadona[df_mercadona.name_stemm==ratios[0]].reference_price.iloc[0].squeeze()
        else:
            x = 0
        exact_price.append(round(x, 3))

    df = pd.DataFrame({
        'originals': lentejas_estofadas_lst,
        'lemma_recipe': list1,
        'potential matches': matches,
        'probability of match': probabilities,
        'original_name_stemm': name,
        'price_stemm': price,
        'price_stemm_gr': exact_price
    }

    )

    return df

In [69]:
df_stemm = match_lists_stemm(recipe_stemm, df_mercadona['name_stemm'].tolist(), lentejas_estofadas_lst)
df_stemm

,originals,lemma_recipe,potential matches,probability of match,original_name_stemm,price_stemm,price_stemm_gr
0,350 g de lentejas castellanas secas,lentejcastellansec,castañ,72,Castañas,3.99,1.396
1,50 g de pimiento rojo en trozos 3-4 cm,pimientrojtroz,piñ,90,Piña,2.25,0.068
2,50 g de pimiento verde en trozos 3-4 cm,pimientverdtroz,piñ,90,Piña,2.25,0.068
3,50 g de zanahoria en trozos 3-4 cm,zanahoritroz,zanahori,90,Zanahorias,0.85,0.042
4,50 g de cebolleta en trozos 3-4 cm,cebollettroz,ceboll,90,Cebollas,1.35,0.068
5,o bien 50 g de cebolla en trozos 3-4 cm,cebolltroz,ceboll,90,Cebollas,1.35,0.068
6,25 g de aceite de oliva,aceitoliv,aceitolivvirgextraeminent,90,Aceite de oliva virgen extra Eminente,13.00,0.065
7,1 cucharadita de sal,cucharadit,turronchocolatsuchard,64,Turrón de chocolate Suchard,3.30,0.063
8,1 cucharadita de pimentón,cucharaditpimenton,ment,90,Menta,1.25,0.031
9,1 pastilla de caldo de pollo,pastillcaldpoll,poll,90,Medio pollo,3.15,0.000


In [70]:
print('El precio de esta receta es de:', round(df_stemm['price_stemm_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra de esta receta es:', round(df_stemm['price_stemm'].sum(), 2), '€')

El precio de esta receta es de: 10.82 €
El precio de la cesta de la compra de esta receta es: 36.94 €


In [51]:
recipe_stemm_space

['lentej castellan sec',
 'pimient roj troz',
 'pimient verd troz',
 'zanahori troz',
 'cebollet troz',
 'ceboll troz',
 'aceit oliv',
 'cucharadit',
 'cucharadit pimenton',
 'pastill cald poll',
 'choriz rodaj',
 'jamon cur dad',
 'agu']

In [71]:
def match_lists_stemm_space(list1, list2, recipe):
    matches = []
    probabilities = []
    name = []
    price = []
    exact_price = []

    for i in list1:
        ratios = process.extractOne(i, list2)
        ratios_recipe = process.extractOne(i, recipe)

        if int(ratios[1]) > 50: # can play with this number
            matches.append(ratios[0])
            probabilities.append(ratios[1])
        else:
            matches.append('no match')
            probabilities.append('probability was too low')
            
        name.append(df_mercadona[df_mercadona.name_stemm_space==ratios[0]].name.iloc[0])
        price.append(df_mercadona[df_mercadona.name_stemm_space==ratios[0]].price.iloc[0].squeeze())
        
    # buscamos el precio exacto en base al gramaje    
        gr = word_tokenize(ratios_recipe[0], language='spanish')
        if gr[1] == 'g':
            x = (int(gr[0])/1000) * df_mercadona[df_mercadona.name_stemm_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'kg':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_stemm_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'l':
            x = int(gr[0]) * df_mercadona[df_mercadona.name_stemm_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharadita':
            x = (int(gr[0])*0.005) * df_mercadona[df_mercadona.name_stemm_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'cucharada':
            x = (int(gr[0])*0.075) * df_mercadona[df_mercadona.name_stemm_space==ratios[0]].reference_price.iloc[0].squeeze()
        elif gr[1] == 'pellizco':
            x = (int(gr[0])*0.001) * df_mercadona[df_mercadona.name_stemm_space==ratios[0]].reference_price.iloc[0].squeeze()
        else:
            x = 0
        exact_price.append(round(x, 3))

    df = pd.DataFrame({
        'originals': lentejas_estofadas_lst,
        'lemma_recipe': list1,
        'potential matches': matches,
        'probability of match': probabilities,
        'original_name_stemm_space': name,
        'price_stemm_space': price,
        'price_stemm_space_gr': exact_price
    }

    )

    return df

In [72]:
df_stemm_space = match_lists_stemm_space(recipe_stemm_space, df_mercadona['name_stemm_space'].tolist(), lentejas_estofadas_lst)
df_stemm_space

,originals,lemma_recipe,potential matches,probability of match,original_name_stemm_space,price_stemm_space,price_stemm_space_gr
0,350 g de lentejas castellanas secas,lentej castellan sec,anis sec regi,86,Anís seco Regio,7.45,2.608
1,50 g de pimiento rojo en trozos 3-4 cm,pimient roj troz,pimient roj,95,Pimiento rojo,0.81,0.134
2,50 g de pimiento verde en trozos 3-4 cm,pimient verd troz,pimient verd,95,Pimiento verde,0.62,0.114
3,50 g de zanahoria en trozos 3-4 cm,zanahori troz,zanahori,90,Zanahorias,0.85,0.042
4,50 g de cebolleta en trozos 3-4 cm,cebollet troz,ceboll,90,Cebollas,1.35,0.000
5,o bien 50 g de cebolla en trozos 3-4 cm,ceboll troz,ceboll,90,Cebollas,1.35,0.068
6,25 g de aceite de oliva,aceit oliv,aceit oliv virg extra eminent,90,Aceite de oliva virgen extra Eminente,13.00,0.065
7,1 cucharadita de sal,cucharadit,turron chocolat suchard,64,Turrón de chocolate Suchard,3.30,0.063
8,1 cucharadita de pimentón,cucharadit pimenton,ment,90,Menta,1.25,0.031
9,1 pastilla de caldo de pollo,pastill cald poll,cald poll pastill,95,Caldo de pollo en pastillas,0.82,0.000


In [73]:
print('El precio de esta receta es de:', round(df_stemm_space['price_stemm_space_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra de esta receta es:', round(df_stemm_space['price_stemm_space'].sum(), 2), '€')

El precio de esta receta es de: 12.6 €
El precio de la cesta de la compra de esta receta es: 34.95 €


In [81]:
df_merge_compare = pd.merge(px_receta[['originals', 'original_name_exact']], 
                            df_lemma[['originals', 'original_name_lemma']], on=['originals'])
df_merge_compare = pd.merge(df_merge_compare, df_lemma_space[['originals', 'original_name_lemma_space']], on=['originals'])
df_merge_compare = pd.merge(df_merge_compare, df_stemm[['originals', 'original_name_stemm']], on=['originals'])
df_merge_compare = pd.merge(df_merge_compare, df_stemm_space[['originals', 'original_name_stemm_space']], on=['originals'])
df_merge_compare

,originals,original_name_exact,original_name_lemma,original_name_lemma_space,original_name_stemm,original_name_stemm_space
0,350 g de lentejas castellanas secas,Lenteja Hacendado,"Pan con semillas 4,5%",Anís seco Regio,Castañas,Anís seco Regio
1,50 g de pimiento rojo en trozos 3-4 cm,Pimiento rojo,Pimiento rojo,Pimiento rojo,Piña,Pimiento rojo
2,50 g de pimiento verde en trozos 3-4 cm,Pimiento verde,Pimiento verde,Pimiento verde,Piña,Pimiento verde
3,50 g de zanahoria en trozos 3-4 cm,Zanahorias,Zanahorias,Zanahorias,Zanahorias,Zanahorias
4,50 g de cebolleta en trozos 3-4 cm,Cebollas dulces,Cebollas,Pollo entero filetes y trozos,Cebollas,Cebollas
5,o bien 50 g de cebolla en trozos 3-4 cm,Cebollas,Cebollas,Cebollas,Cebollas,Cebollas
6,25 g de aceite de oliva,Aceite de oliva virgen Hacendado,Aceite de oliva virgen extra Eminente,Aceite de oliva virgen extra Eminente,Aceite de oliva virgen extra Eminente,Aceite de oliva virgen extra Eminente
7,1 cucharadita de sal,Sal fina Hacendado,Yuca,Yuca,Turrón de chocolate Suchard,Turrón de chocolate Suchard
8,1 cucharadita de pimentón,Pimentón dulce Hacendado,Menta,Queso madurado de cabra con pimentón de la Ver...,Menta,Menta
9,1 pastilla de caldo de pollo,Caldo de pollo Hacendado en pastillas,Medio pollo,Caldo de pollo en pastillas,Medio pollo,Caldo de pollo en pastillas


In [86]:
print('El precio por el método exacto es de:', round(px_receta['price_exact_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra por el método exacto es:', round(px_receta['price_exact'].sum(), 2), '€ \n'), 

print('El precio por el método lemma es de:', round(df_lemma['price_lemma_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra por el método lemma es:', round(df_lemma['price_lemma'].sum(), 2), '€ \n')

print('El precio por el método lemma_space es de:', round(df_lemma_space['price_lemma_space_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra por el método lemma_space es:', round(df_lemma_space['price_lemma_space'].sum(), 2), '€ \n')

print('El precio por el método stemm es de:', round(df_stemm['price_stemm_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra de esta receta es:', round(df_stemm['price_stemm'].sum(), 2), '€ \n')

print('El precio por el método stemm_space es de:', round(df_stemm_space['price_stemm_space_gr'].sum(), 2), '€')
print('El precio de la cesta de la compra por el método stemm_space es:', round(df_stemm_space['price_stemm_space'].sum(), 2), '€ \n')

El precio por el método exacto es de: 2.19 €
El precio de la cesta de la compra por el método exacto es: 17.71 € 

El precio por el método lemma es de: 3.32 €
El precio de la cesta de la compra por el método lemma es: 30.57 € 

El precio por el método lemma_space es de: 12.51 €
El precio de la cesta de la compra por el método lemma_space es: 42.84 € 

El precio por el método stemm es de: 10.82 €
El precio de la cesta de la compra de esta receta es: 36.94 € 

El precio por el método stemm_space es de: 12.6 €
El precio de la cesta de la compra por el método stemm_space es: 34.95 € 

